# Conversion of TXT to MAT for gm/ID result files for GF180MCU

**(c) 2024-2025 Boris Murmann and Harald Pretl**

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0

In [20]:
import pandas as pd
import numpy as np
from numpy._core.records import fromarrays
from scipy.io import savemat

In [21]:
choice = 0  #start from 0
devices = ['gf180mcu_nmos_06v0', 'gf180mcu_pmos_06v0']

# widths used for characterization and fringe cap parameters (fringe caps are not included in ngspice output)
w = np.array([5, 5])
nfing = np.array([1, 1])

In [22]:
df_raw = pd.read_csv('techsweep_'+devices[choice]+'.txt', sep=r'\s+')
par_names = df_raw.columns.to_list()
par_prefix = par_names[1].split('[')[0]

In [23]:
# remove extra headers in file body and unwanted columns
df = df_raw.drop(['frequency', 'frequency.1'], axis=1)
df = df.apply(pd.to_numeric)

# rename columns for readability
df.columns = df.columns.str.removeprefix(par_prefix+'[')
df.columns = df.columns.str.replace(par_prefix[1:], '')
df.columns = df.columns.str.removesuffix(']')

# round sweep vectors to easily addressable values
df['l'] = df['l'].apply(lambda x: round(x*1e6, 2))
df['vgs'] = df['vgs'].apply(lambda x: round(x, 2))
df['vds'] = df['vds'].apply(lambda x: round(x, 2))
df['vbs'] = df['vbs'].apply(lambda x: round(x, 2))
df

,cdd,cgb,cgd,cgdo,cgg,cgs,cgso,css,gds,gm,gmbs,id,l,vbs,vds,vdsat,vgs,vth
0,2.457000e-20,-2.588000e-15,-2.464000e-20,1.519000e-15,2.591000e-15,-2.296000e-18,1.519000e-15,7.993000e-21,8.344000e-13,0.000000e+00,0.000000e+00,0.000000e+00,0.7,-0.0,-0.00,0.03413,-0.0,0.7234
1,6.045000e-22,-2.210000e-15,-6.390000e-22,1.519000e-15,2.211000e-15,-3.729000e-19,1.519000e-15,1.810000e-22,4.000000e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.7,-0.4,0.00,0.03595,0.0,0.8812
2,2.741000e-23,-1.960000e-15,-2.976000e-23,1.519000e-15,1.960000e-15,1.525000e-19,1.519000e-15,7.789000e-24,4.427000e-17,0.000000e+00,0.000000e+00,0.000000e+00,0.7,-0.8,0.00,0.03693,0.0,1.0100
3,1.898000e-24,-1.779000e-15,-2.090000e-24,1.519000e-15,1.779000e-15,3.386000e-19,1.519000e-15,5.182000e-25,8.882000e-19,0.000000e+00,0.000000e+00,0.000000e+00,0.7,-1.2,0.00,0.03747,0.0,1.1210
4,1.167000e-21,-2.588000e-15,-1.060000e-21,1.489000e-15,2.591000e-15,-2.312000e-18,1.519000e-15,1.619000e-20,8.766000e-14,4.523000e-13,2.566000e-13,1.757000e-14,0.7,-0.0,0.05,0.03413,-0.0,0.7234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244819,2.246000e-17,-8.878000e-16,-2.330000e-17,1.545000e-15,7.454000e-14,-7.363000e-14,1.614000e-15,5.423000e-14,4.039000e-07,1.039000e-04,3.278000e-05,2.469000e-04,10.0,-1.2,4.95,3.30800,5.0,1.1340
244820,2.248000e-17,9.188000e-15,-2.058000e-17,1.519000e-15,7.799000e-14,-8.716000e-14,1.614000e-15,6.180000e-14,4.689000e-07,1.127000e-04,4.970000e-05,2.955000e-04,10.0,-0.0,5.00,3.46100,5.0,0.6741
244821,2.293000e-17,3.075000e-15,-2.224000e-17,1.519000e-15,7.647000e-14,-7.952000e-14,1.614000e-15,5.806000e-14,4.428000e-07,1.091000e-04,4.291000e-05,2.769000e-04,10.0,-0.4,5.00,3.41700,5.0,0.8517
244822,2.214000e-17,4.579000e-16,-2.230000e-17,1.519000e-15,7.539000e-14,-7.583000e-14,1.614000e-15,5.579000e-14,4.186000e-07,1.063000e-04,3.723000e-05,2.609000e-04,10.0,-0.8,5.00,3.36400,5.0,1.0020


In [24]:
# sweep variable vectors
l = np.unique(abs(df['l']))
vgs = np.unique(abs(df['vgs']))
vds = np.unique(abs(df['vds']))
vbs = np.unique(abs(df['vbs']))

In [25]:
# data
# ngspice sweep order is l, vgs, vds, vbs
dims = [len(l), len(vgs), len(vds), len(vbs)]
print(dims)
id = np.reshape(df['id'].values, dims, order='C')
vt = np.reshape(df['vth'].values, dims, order='C')
gm = np.reshape(df['gm'].values, dims, order='C')
gmbs = np.reshape(df['gmbs'].values, dims, order='C')
gds = np.reshape(df['gds'].values, dims, order='C')
cgg = np.reshape(df['cgg'].values, dims, order='C') \
      + np.reshape(df['cgdo'].values, dims, order='C') \
      + np.reshape(df['cgso'].values, dims, order='C')
cgb = -np.reshape(df['cgb'].values, dims, order='C')
cgd = -np.reshape(df['cgd'].values, dims, order='C') \
      + np.reshape(df['cgdo'].values, dims, order='C')
cgs = -np.reshape(df['cgs'].values, dims, order='C') \
      + np.reshape(df['cgso'].values, dims, order='C')
cdd = np.reshape(df['cdd'].values, dims, order='C') \
      + np.reshape(df['cgdo'].values, dims, order='C')
css = np.reshape(df['css'].values, dims, order='C') \
      + np.reshape(df['cgso'].values, dims, order='C')
#sth = np.reshape(df['sid'].values, dims, order='C')
#sfl = np.reshape(df['sfl'].values, dims, order='C')

[6, 101, 101, 4]


In [26]:
dic = {
  "INFO": "GLOFO GF180MCU, 180nm CMOS, BSIM4",
  "CORNER": "NOM",
  "TEMP": 300.0,
  "VGS": vgs,
  "VDS": vds,
  "VSB": vbs,
  "L": l,
  "W": w[choice],
  "NFING": nfing[choice],
  "ID": id,
  "VT": vt,
  "GM": gm,
  "GMB": gmbs,
  "GDS": gds,
  "CGG": cgg,
  "CGB": cgb,
  "CGD": cgd,
  "CGS": cgs,
  "CDD": cdd,
  "CSS": css,
  "STH": 0.0,
  "SFL": 0.0
}

dict

savemat(devices[choice]+'.mat', {devices[choice]: dic})